In [1]:
import os
import sys

project_directory = os.path.abspath('..')
sys.path.append(project_directory)

In [2]:
import torch
import torch.nn as nn
from src.utils.common import set_seed

set_seed(42)

In [3]:
from src.dataloaders.dataloader import create_dataloaders
train_loader, val_loader, test_loader = create_dataloaders(data_root='../data')

In [4]:
train_dataset, val_dataset, test_dataset = create_dataloaders(data_root='../data', return_datasets=True)

In [5]:
from src.models.modern_cnn_v1 import ModernCNNv1

In [6]:
from torch.utils.data import Subset, DataLoader

train_subset = Subset(test_dataset, range(64))
val_subset   = Subset(val_dataset, range(64))

In [7]:
train_loader_small = DataLoader(
    train_subset,
    batch_size=16,
    shuffle=True
)

val_loader_small = DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False
)

In [8]:
model = ModernCNNv1()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
from src.train.trainer import fit, EarlyStopper

In [11]:
early_stopper = EarlyStopper(patience=2)

test_result = fit(
    model=model,
    train_loader=train_loader_small,
    val_loader=val_loader_small,
    optimizer=optimizer,
    criterion=criterion,
    device=DEVICE, 
    epochs=10,
    early_stopper=early_stopper
)

EPOCH 1/10 | Train Loss: 2.880077 Acc: 0.140625 | Val Loss: 2.305816 Acc: 0.095800 
EPOCH 2/10 | Train Loss: 2.295149 Acc: 0.328125 | Val Loss: 2.324074 Acc: 0.122000 
EPOCH 3/10 | Train Loss: 1.949152 Acc: 0.375000 | Val Loss: 2.376952 Acc: 0.105200 
Early stop at epoch 3 | Best epoch 1


In [12]:
test_result

{'train_loss': [2.8800772428512573, 2.2951489090919495, 1.9491521418094635],
 'val_loss': [2.3058158504486084, 2.3240738567352297, 2.376951912689209],
 'train_acc': [0.140625, 0.328125, 0.375],
 'val_acc': [0.0958, 0.122, 0.1052]}